# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,urangan,-25.3000,152.9000,30.67,72,5,2.24,AU,1706916896
1,1,albany,42.6001,-73.9662,1.72,79,91,0.45,US,1706916896
2,2,west island,-12.1568,96.8225,27.99,83,40,4.63,CC,1706916812
3,3,gadzhiyevo,69.2551,33.3362,-10.54,91,66,6.10,RU,1706916897
4,4,labrador city,52.9463,-66.9114,-23.05,68,0,1.03,CA,1706916897


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display


# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) &
                                 (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
narrow_city_df = city_data_df.dropna()

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,urangan,-25.3000,152.9000,30.67,72,5,2.24,AU,1706916896
1,1,albany,42.6001,-73.9662,1.72,79,91,0.45,US,1706916896
2,2,west island,-12.1568,96.8225,27.99,83,40,4.63,CC,1706916812
3,3,gadzhiyevo,69.2551,33.3362,-10.54,91,66,6.10,RU,1706916897
4,4,labrador city,52.9463,-66.9114,-23.05,68,0,1.03,CA,1706916897
...,...,...,...,...,...,...,...,...,...,...
558,558,rovnoye,50.7765,46.0509,-3.16,84,33,7.45,RU,1706916990
559,559,yinchuan,38.4681,106.2731,-3.12,88,99,1.94,CN,1706916990
560,560,nizhniy kuranakh,58.8353,125.4869,-27.97,100,100,1.53,RU,1706916991
561,561,ierapetra,35.0100,25.7353,12.30,61,21,3.58,GR,1706916991


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,urangan,AU,-25.3000,152.9000,72,
1,albany,US,42.6001,-73.9662,79,
2,west island,CC,-12.1568,96.8225,83,
3,gadzhiyevo,RU,69.2551,33.3362,91,
4,labrador city,CA,52.9463,-66.9114,68,
...,...,...,...,...,...,...
558,rovnoye,RU,50.7765,46.0509,84,
559,yinchuan,CN,38.4681,106.2731,88,
560,nizhniy kuranakh,RU,58.8353,125.4869,100,
561,ierapetra,GR,35.0100,25.7353,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
urangan - nearest hotel: Ramada
albany - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
gadzhiyevo - nearest hotel: Оазис
labrador city - nearest hotel: Carol Inn
port-aux-francais - nearest hotel: Keravel
longyearbyen - nearest hotel: The Vault
charters towers - nearest hotel: No hotel found
vorgashor - nearest hotel: София
grytviken - nearest hotel: No hotel found
hastings - nearest hotel: White Rock Hotel
ormond-by-the-sea - nearest hotel: Howard Johnson
barra - nearest hotel: No hotel found
rechka-vydrino - nearest hotel: Сердце Мамая
bethel - nearest hotel: Hampton Inn Danbury
unai - nearest hotel: Hotel Fazenda Curva do Rio
huarmey - nearest hotel: Hostal Santa Rosa
puerto natales - nearest hotel: Dorotea Patagonia Hostel
utrik - nearest hotel: No hotel found
nouna - nearest hotel: No hotel found
baiti - nearest hotel: No hotel found
ancaster - nearest hotel: The Barracks Inn
muli - nearest hotel: Mulaku Atoll
sur - neares

saint-joseph - nearest hotel: PALM Hotel & Spa
yaren - nearest hotel: Aiwo Hotel
sinabang - nearest hotel: No hotel found
plainview - nearest hotel: Residence Inn by Marriott
jianshui - nearest hotel: 阳阳大酒店
longmen - nearest hotel: No hotel found
gaalkacyo - nearest hotel: Citycenter وسط المدينة
talnakh - nearest hotel: Талнах
newman - nearest hotel: No hotel found
zawiercie - nearest hotel: Hotel Zawiercie
nabire - nearest hotel: Hotel Anggrek
broome - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
klyuchi - nearest hotel: No hotel found
hibbing - nearest hotel: Hibbing Park Hotel
enoch - nearest hotel: Travelodge by Wyndham Cedar City
karachev - nearest hotel: Гостиница "Карачевъ"
ust'-kut - nearest hotel: Домашний очаг
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
pleiku - nearest hotel: Green Bamboo - Tre Xanh Plaza Hotel
road town - nearest hotel: No hotel found
babayevo - nearest hotel

hammerfest - nearest hotel: Thon Hotel Hammerfest
novodvinsk - nearest hotel: Уют
acapulco de juarez - nearest hotel: Hotel del Valle
borogontsy - nearest hotel: No hotel found
northam - nearest hotel: Durrant House Hotel
muang phonsavan - nearest hotel: COECCO Xieng Khouang Hotel
leeton - nearest hotel: No hotel found
gizo - nearest hotel: Gizo Hotel
codrington - nearest hotel: No hotel found
sorong - nearest hotel: Hotel Membramo Marina
tommot - nearest hotel: Томмот
rongelap - nearest hotel: No hotel found
kiunga - nearest hotel: No hotel found
tafuna - nearest hotel: Tradewinds
khandyga - nearest hotel: Апельсин
banket - nearest hotel: No hotel found
shizuishan - nearest hotel: No hotel found
arlington heights - nearest hotel: Hampton Inn and Suites- Mount Prospect
hyesan-dong - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
kataysk - nearest hotel: No hotel found
inyonga - nearest hotel: Mikumi Lodge
yenotayevka - nearest hotel: No hotel found
ad dilam - 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,urangan,AU,-25.3000,152.9000,72,Ramada
1,albany,US,42.6001,-73.9662,79,No hotel found
2,west island,CC,-12.1568,96.8225,83,Cocos Village Bungalows
3,gadzhiyevo,RU,69.2551,33.3362,91,Оазис
4,labrador city,CA,52.9463,-66.9114,68,Carol Inn
...,...,...,...,...,...,...
558,rovnoye,RU,50.7765,46.0509,84,No hotel found
559,yinchuan,CN,38.4681,106.2731,88,Yinchuan hotel
560,nizhniy kuranakh,RU,58.8353,125.4869,100,Самородок
561,ierapetra,GR,35.0100,25.7353,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                    geo = True,
                                    color = "City",
                                    alpha = 0.8,
                                    size = "Humidity",
                                    tiles = "OSM",
                                    hover_cols=["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)